In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import odeint
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import proj3d

%matplotlib qt

# 1. Data Generation (Start slightly off-center to trigger the chaos)
def lorenz(state, t, sigma=10, beta=8/3, rho=28):
    x, y, z = state
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

t = np.linspace(0, 40, 5000)
sol = odeint(lorenz, [0.5, 0.5, 10], t) 
x_val, y_val, z_val = sol.T

# 2. Reconstruct Shadow Manifold Mx
tau = 25 
mx_x, mx_y, mx_z = x_val[2*tau:], x_val[tau:-tau], x_val[:-2*tau]
m_x, m_y, m_z = x_val[2*tau:], y_val[2*tau:], z_val[2*tau:]

# 3. Figure Setup
fig = plt.figure(figsize=(15, 8))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

def setup_ax(ax, data_x, data_y, data_z, title):
    # Calculate span
    ranges = [data_x.max()-data_x.min(), data_y.max()-data_y.min(), data_z.max()-data_z.min()]
    # Add a tiny epsilon (0.1) to max_range to prevent the "singular" error if data is a single point
    max_range = max(max(ranges) / 2.0, 0.1) 
    
    mid_x, mid_y, mid_z = np.mean(data_x), np.mean(data_y), np.mean(data_z)
    
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    ax.set_title(title, fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.xaxis.pane.fill = ax.yaxis.pane.fill = ax.zaxis.pane.fill = False

setup_ax(ax1, m_x, m_y, m_z, r"Original Manifold $M$")
setup_ax(ax2, mx_x, mx_y, mx_z, r"Shadow Manifold $M_x$")

# Artists
line1, = ax1.plot([], [], [], lw=0.8, color='blue', alpha=0.5)
line2, = ax2.plot([], [], [], lw=0.8, color='green', alpha=0.5)
dot1,  = ax1.plot([], [], [], 'ro', markersize=6, zorder=10)
dot2,  = ax2.plot([], [], [], 'ro', markersize=6, zorder=10)

link = Line2D([0, 0], [0, 0], color='red', linestyle='--', lw=1.2, alpha=0.8, transform=fig.transFigure)
fig.lines.append(link)

def get_2d_coord(ax, p3d):
    x_s, y_s, z_s = proj3d.proj_transform(p3d[0], p3d[1], p3d[2], ax.get_proj())
    x_d, y_d = ax.transData.transform((x_s, y_s))
    return fig.transFigure.inverted().transform((x_d, y_d))

def update(frame):
    idx = frame * 2
    if idx >= len(m_x): idx = len(m_x) - 1
    
    line1.set_data(m_x[:idx], m_y[:idx])
    line1.set_3d_properties(m_z[:idx])
    line2.set_data(mx_x[:idx], mx_y[:idx])
    line2.set_3d_properties(mx_z[:idx])
    
    dot1.set_data([m_x[idx]], [m_y[idx]])
    dot1.set_3d_properties([m_z[idx]])
    dot2.set_data([mx_x[idx]], [mx_y[idx]])
    dot2.set_3d_properties([mx_z[idx]])
    
    ax1.view_init(elev=20, azim=45 + frame * 0.15)
    ax2.view_init(elev=20, azim=45 + frame * 0.15)
    
    p1_2d = get_2d_coord(ax1, (m_x[idx], m_y[idx], m_z[idx]))
    p2_2d = get_2d_coord(ax2, (mx_x[idx], mx_y[idx], mx_z[idx]))
    link.set_xdata([p1_2d[0], p2_2d[0]])
    link.set_ydata([p1_2d[1], p2_2d[1]])
    
    return line1, line2, dot1, dot2, link

ani = FuncAnimation(fig, update, frames=len(m_x)//2, interval=50, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.animation import PillowWriter

# 1. Define the steady-state section (skip the initial spiral from 0,0,0)
# Assuming 5000 total points, starting at 1500 skips the transient.
t_start = 1500 
t_end = 3500   

# We use a step of 10 to keep the frame count around 200
# (3500 - 1500) / 10 = 200 frames
save_range = range(t_start // 2, t_end // 2, 5) 

# 2. Update the animation object with the specific range
ani = FuncAnimation(fig, update, frames=save_range, interval=50, blit=False)

# 3. Save with optimized settings for GitHub
print("Encoding optimized GIF for GitHub...")
# Set a slightly lower DPI to keep file size small
writer = PillowWriter(fps=20)
ani.save("lorenz_takens_reconstruction_new.gif", writer=writer, dpi=80)
print("Done! Check your folder.")

Encoding optimized GIF for GitHub...
Done! Check your folder.
